In [2]:
import csv
import itertools
import operator
import numpy as np
import nltk
import sys
from datetime import datetime
from utils import *
from rnn_theano import RNNTheano
from gru_theano import GRUTheano
import time

import matplotlib.pyplot as plt
%matplotlib inline


In [ ]:
x=np.load('xtrain.npy')
y=np.load('ytrain.npy')

In [3]:
model = RNNTheano(8000, hidden_dim=80)
t1 = time.time()
model.sgd_step(x[10], y[10], 0.005)
t2 = time.time()
print("SGD Step time: %f milliseconds" % ((t2 - t1) * 1000.))

In [4]:
model = GRUTheano(8000, hidden_dim=80)
t1 = time.time()
model.sgd_step(x[10], y[10], 0.005)
t2 = time.time()
print("SGD Step time: %f milliseconds" % ((t2 - t1) * 1000.))

/Users/ankurgoel/kaggle/datastuff/nn/gru_theano.py:112: UserWarning: The Param class is deprecated. Replace Param(default=N) by theano.In(value=N)
  [x, y, learning_rate, theano.Param(decay, default=0.9)],


SGD Step time: 418.977022 milliseconds


In [9]:
vocabulary_size = 8000
unknown_token = "UNKNOWN_TOKEN"
sentence_start_token = "SENTENCE_START"
sentence_end_token = "SENTENCE_END"
 
# Read the data and append SENTENCE_START and SENTENCE_END tokens
print ("Reading CSV file...")
with open('reddit-comments-2015-08.csv', 'rt',encoding='utf-8') as f:
    reader = csv.reader(f, skipinitialspace=True)
    reader.__next__()
    # Split full comments into sentences
    sentences = itertools.chain(*[nltk.sent_tokenize(x[0].lower()) for x in reader])
    # Append SENTENCE_START and SENTENCE_END
    sentences = ["%s %s %s" % (sentence_start_token, x, sentence_end_token) for x in sentences]
print ("Parsed %d sentences." % (len(sentences)))
     
# Tokenize the sentences into words
tokenized_sentences = [nltk.word_tokenize(sent) for sent in sentences]
 
# Count the word frequencies
word_freq = nltk.FreqDist(itertools.chain(*tokenized_sentences))
print ("Found %d unique words tokens." % len(word_freq.items()))
 
# Get the most common words and build index_to_word and word_to_index vectors
vocab = word_freq.most_common(vocabulary_size-1)
index_to_word = [x[0] for x in vocab]
index_to_word.append(unknown_token)
word_to_index = dict([(w,i) for i,w in enumerate(index_to_word)])
 
print ("Using vocabulary size %d." % vocabulary_size)
print ("The least frequent word in our vocabulary is '%s' and appeared %d times." % (vocab[-1][0], vocab[-1][1]))
 
# Replace all words not in our vocabulary with the unknown token
for i, sent in enumerate(tokenized_sentences):
    tokenized_sentences[i] = [w if w in word_to_index else unknown_token for w in sent]
 
print ("\nExample sentence: '%s'" % sentences[0])
print ("\nExample sentence after Pre-processing: '%s'" % tokenized_sentences[0])
X_train = np.asarray([[word_to_index[w] for w in sent[:-1]] for sent in tokenized_sentences])
y_train = np.asarray([[word_to_index[w] for w in sent[1:]] for sent in tokenized_sentences])
np.save('xtrain',X_train)
np.save('ytrain',y_train)

Reading CSV file...
Parsed 79170 sentences.
Found 65752 unique words tokens.
Using vocabulary size 8000.
The least frequent word in our vocabulary is 'recreational' and appeared 10 times.

Example sentence: 'SENTENCE_START i joined a new league this year and they have different scoring rules than i'm used to. SENTENCE_END'

Example sentence after Pre-processing: '['SENTENCE_START', 'i', 'joined', 'a', 'new', 'league', 'this', 'year', 'and', 'they', 'have', 'different', 'scoring', 'rules', 'than', 'i', "'m", 'used', 'to', '.', 'SENTENCE_END']'
